<a href="https://colab.research.google.com/github/Fortune-Adekogbe/VisualPlagiarism/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/MyDrive/Projects/VisualPlagiarism

/content/drive/MyDrive/Projects/VisualPlagiarism


In [ ]:
!ls data

 BLIP	       jpeg	   'Plagiarised images'		   'Plagiarised Images UI - Sheet1.csv'
 CLIP	       MetaCLIP    'Plagiarised images 1'	    sigLIP
 CS_Detector   MobileCLIP  'Plagiarised Images UI.gsheet'


## Setup

In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import os
import json
import time
from datetime import date
from google.colab import userdata

## Common Screens

search for design URLs: https://commoncrawl.org/get-started
For using Asyncio with S3 - https://skeptric.com/reading-parquet-metadata/

### Download

In [ ]:
!pip install boto3 -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 8.2 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import boto3
from botocore.exceptions import NoCredentialsError, PartialCredentialsError
from botocore.config import Config
import concurrent.futures

config = Config(
    retries = {
        'max_attempts': 2,
        'mode': 'standard'
    },
    max_pool_connections=25  # Increase the pool size as needed
)

# Create a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
    region_name='us-west-2'
)

# Create an S3 client using the session
s3 = session.client('s3', config=config)

# Specify the bucket name
bucket_name = 'common-screens'

prefix = "data/jpeg/4"
os.makedirs(prefix, exist_ok=True)

max_objects = 200_000

def download_object(obj):
    try:
        if not os.path.exists(obj["Key"]):

            s3.download_file(bucket_name, obj['Key'], obj['Key'].replace('data/jpeg', prefix))
    except Exception as e:
        print(f"Failed to download {obj['Key']}: {e}")

try:
    # Initialize paginator
    paginator = s3.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

    # Counter to limit the number of objects listed
    object_count = 0

    with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        for page in page_iterator:
            if 'Contents' in page:
                futures = []
                for obj in page['Contents']:
                    if object_count >= max_objects:
                        break
                    futures.append(executor.submit(download_object, obj))
                    object_count += 1
                # Ensure all futures are completed
                concurrent.futures.wait(futures)
            if object_count >= max_objects:
                break

except (NoCredentialsError, PartialCredentialsError):
    print("Credentials are not available or incomplete.")

In [ ]:
import boto3
# Create a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY'),
    region_name='us-west-2'
)
bucket_name = 'common-screens'

s3 = session.client('s3') # Directly create the s3 client
paginator = s3.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=f"data/jpeg/22")

# Counter to limit the number of objects listed
object_count = 0
max_objects = 10

tasks = []
for page in page_iterator:
    if 'Contents' in page:
        for obj in page['Contents']:
            if object_count >= max_objects:
                break
            print(obj["Key"])
            object_count += 1
    if object_count >= max_objects:
        break

### Data Cleaning

1. Handle repititions. Like the google sign in page.
2. Handle adult content pages. Like the one at sorted index 104,000. ✅

In [ ]:
import os
import concurrent.futures
from tqdm import tqdm
import shutil

def move_file(file_):
    src_file = os.path.join(src_folder, file_)
    dst_file = os.path.join(dst_folder, file_)
    shutil.move(src_file, dst_file)

def merge_folders():
    with concurrent.futures.ProcessPoolExecutor(max_workers=24) as executor:
        futures = []
        for filename in os.listdir(src_folder)[:283]:
            futures.append(executor.submit(move_file, filename))

        # Wrap the futures with tqdm to create a progress bar
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            result = future.result()

# Example usage
src_folder = "data/jpeg/3"
dst_folder = "data/jpeg"
merge_folders()

In [ ]:
prefix = "data/jpeg"

In [ ]:
import os

list(os.scandir('data'))

In [ ]:
filenames = os.scandir(prefix)

In [ ]:
filenames = list(filenames)

In [ ]:
100_000 - len(filenames)

In [ ]:
filenames[-10:]

In [ ]:
names = sorted([x.name for x in filenames if x.name.endswith("jpeg")])

In [ ]:
names[-10:]

#### Remove Similar Images?

In [ ]:
from collections import defaultdict

unik = defaultdict(list)

for i in filenames:
    if i.name.endswith("jpeg"):
        unik[i.name.split('-', 1)[1]].append(i.path)

In [ ]:
sum([1 for i in unik if len(unik[i]) > 1])

In [ ]:
unik

In [ ]:
from PIL import Image
img = Image.open('data/jpeg/1007-5462-info--.jpeg')
w, h = img.size
img.resize((w//4, h//4))

In [ ]:
from PIL import Image
img = Image.open('data/jpeg/0204-5462-info--.jpeg')
w, h = img.size
img.resize((w//4, h//4))

In [ ]:
import cv2

def get_mean_variance(image_path):
    img = cv2.imread(image_path)
    return (img.mean(), img.var()), image_path

get_mean_variance('data/jpeg/0204-5462-info--.jpeg')

In [ ]:
get_mean_variance('data/jpeg/1007-5462-info--.jpeg')

In [ ]:
get_mean_variance('data/jpeg/1000craft-blogspot-com--.jpeg')

In [ ]:
for i in filenames:
    if 'blogspot' in i.name.lower():
        print(i.name)

In [ ]:
filenames[0]

In [ ]:
# @title
# import concurrent

# with concurrent.futures.ThreadPoolExecutor(max_workers=24) as executor:
#     futures = []
#     for i in filenames:
#         futures.append(executor.submit(get_mean_variance, i.path))
#     # Ensure all futures are completed
#     concurrent.futures.wait(futures)

#     results = defaultdict(list)
#     for future in futures:
#         try:
#             mv, ip = future.result()
#             results[mv].append(ip)
#         except Exception as e:
#             print(f"An error occurred: {e}")

In [ ]:
import concurrent.futures
from tqdm import tqdm
from collections import defaultdict

# Assume get_mean_variance is already defined

results = defaultdict(list)

with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
    futures = []
    for i in filenames:
        futures.append(executor.submit(get_mean_variance, i.path))

    # Wrap the futures with tqdm to create a progress bar
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        try:
            mv, ip = future.result()
            results[mv].append(ip)
        except Exception as e:
            print(f"An error occurred: {e}")

In [ ]:
S = 0
for i in results:
    if len(results[i]) > 1:
        for j in results[i][1:]:
            try:
                os.remove(j)
            except:
                S += 1
print(S)

#### Remove Blank

In [ ]:
def is_mv_blank(mean_value, var_value, threshold_mean=250, threshold_var=10):
    return mean_value > threshold_mean and var_value < threshold_var
S = 0

for i in results:
    if is_mv_blank(i[0], i[1]):
        print(i)
        for j in results[i]:
            try:
                os.remove(j)
                S += 1
            except:
                ...
print(S)

In [ ]:
import os, shutil
from PIL import Image
import numpy as np
from multiprocessing import Pool

def is_blank(image_path, threshold_mean=250, threshold_var=10):
    try:
        with Image.open(image_path) as img:
            img = img.convert('L')  # Convert to grayscale
            img_array = np.array(img)
            mean_value = np.mean(img_array)
            var_value = np.var(img_array)
            return mean_value > threshold_mean and var_value < threshold_var
    except Exception as e:
        return False

def process_images(image_paths, output_folder):
    blank_images = []
    for image_path in tqdm(image_paths):
        if is_blank(image_path):
            blank_images.append(image_path)
            dest_path = os.path.join(output_folder, os.path.basename(image_path))
            shutil.move(image_path, dest_path)
            # os.remove(image_path)
    return blank_images

def main(image_folder, output_folder, num_workers=16):
    image_paths = [os.path.join(image_folder, fname) for fname in filenames]

    pool = Pool(num_workers)
    chunk_size = len(image_paths) // num_workers
    chunks = [image_paths[i:i + chunk_size] for i in range(0, len(image_paths), chunk_size)]

    results = pool.starmap(process_images, [(chunk, output_folder) for chunk in chunks])
    pool.close()
    pool.join()

    blank_images = [img for sublist in results for img in sublist]
    return blank_images

if __name__ == "__main__":
    image_folder = 'data/jpeg'
    output_folder = 'data/blank'
    blank_images = main(image_folder, output_folder)
    print(f"Moved {len(blank_images)} blank or nearly blank images.")

#### Remove Explicit

In [ ]:
checklist = ["sex", "girls", "xxx", "adult", "nipples", "porn", "nudity", "nude", "mature", "slut", "lesbi", "blowjob", "loli", "jav", "hentai", "brazzers", "casino", ]

In [ ]:
explicit = [i for i in filenames if any(j in i.name.lower() for j in checklist)]

In [ ]:
len(explicit)

In [ ]:
for i in explicit:
    try:
        os.remove(i.path)
    except:
        ...

### Data Splitting

In [ ]:
!ls data

In [ ]:
from sklearn.model_selection import train_test_split
import json
import  os

filenames = os.scandir('data/jpeg')
filenames = [i.path for i in filenames]

train_filenames, test_filenames = train_test_split(filenames, test_size=0.2, random_state=42)
train_filenames, val_filenames = train_test_split(train_filenames, test_size=0.25, random_state=42)

data_splits = {
    'train': train_filenames,
    'val': val_filenames,
    'test': test_filenames
}

os.makedirs('data/CS_Detector', exist_ok=True)

json.dump(data_splits, open('data/CS_Detector/data_splits.json', 'w'))

### Data Pairing

In [ ]:
import json
import os

data_splits = json.load(open('data/CS_Detector/data_splits.json'))  # Load the JSON file

In [ ]:
filenames = os.scandir('data/jpeg')

In [ ]:
import numpy as np
import math
import random
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from tqdm.auto import tqdm
import os, shutil
from PIL import Image
import cv2
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging
import gc

# Set up logging
logging.basicConfig(filename='error.log', level=logging.ERROR)


np.random.seed(42)
random.seed(42)

# Initialize the data generator for augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

def generate_augmented_images(image, num_augmentations):
    image = image.reshape((1,) + image.shape)  # Reshape for the generator
    augmented_images = []
    i = 0
    for batch in datagen.flow(image, batch_size=1):
        augmented_images.append(batch[0])
        i += 1
        if i >= num_augmentations:
            break
    return np.array(augmented_images)

# Generate augmented pairs for each image

def generate_image_pairs(image, images, num_augmentations=2):
    try:
        positive_pairs = []
        negative_pairs = []

        img = Image.open(image)
        img = img.convert('RGB')
        img = img.resize((598, 336))
        img = np.array(img)

        positive_pairs.append([img, img, 1])  # Pair original with original
        augmented_images = generate_augmented_images(img, num_augmentations)

        for i, aug_img in enumerate(augmented_images):
            positive_pairs.append([img, aug_img, 1])  # Pair original with augmented
            for j in range(i+1, num_augmentations):
                other_aug_img = augmented_images[j]
                if not np.array_equal(aug_img, other_aug_img):
                    positive_pairs.append([aug_img, other_aug_img, 1])  # Pair augmented with augmented
        # Create negative pairs (img1, img2)
        N = math.comb(num_augmentations + 1, 2) + 1

        for neg_image in random.sample(images, N):
            if neg_image != image:
                neg_img = Image.open(neg_image)
                neg_img = neg_img.convert('RGB')
                neg_img = neg_img.resize((598, 336))
                neg_img = np.array(neg_img)
                negative_pairs.append((img, neg_img, 0))
            if len(negative_pairs) == len(positive_pairs):
                break
        # Combine positive and negative pairs
        all_pairs = positive_pairs + negative_pairs
        # random.shuffle(all_pairs)
        return all_pairs
    except Exception as e:
        logging.error(f"Error generating pairs for image {image}: {e}")
        return []

a = generate_image_pairs('data/jpeg/1000kanalov-ru--.jpeg', ['data/jpeg/1000kanalov-ru--.jpeg']*20)

In [ ]:
from matplotlib import pyplot as plt
plt.axis('off')
plt.imshow(a[0][0])

In [ ]:
!ls data/CS_Detector

In [ ]:
import h5py

def add_item_to_hdf5(filename, new_data):
    with h5py.File(filename, 'a') as hf:
        # Determine the next available index for the new item
        item_index = len(hf.keys())

        for pair in new_data:
            arr1, arr2, label = pair
            group = hf.create_group(f'item_{item_index}')
            group.create_dataset('image1', data=arr1, compression="gzip", compression_opts=9)
            group.create_dataset('image2', data=arr2, compression="gzip", compression_opts=9)
            group.attrs['label'] = label
            item_index += 1

def generate_pairs(split, data_splits, n_workers=4, num_augmentations=2):
    n_subset = len(data_splits[split])//8 + 32
    images = data_splits[split][:n_subset]

    # with ThreadPoolExecutor(max_workers=n_workers) as executor:
        # futures = [executor.submit(generate_image_pairs, img, images, num_augmentations) for img in images]
        # i = 0
    filename = f'data/CS_Detector/{split}_pairs.h5'
    # with h5py.File(f'data/CS_Detector/{split}_pairs.h5', 'w') as hf:
    # for future in tqdm(as_completed(futures), total=len(futures)):
    for img in tqdm(images):
        result = generate_image_pairs(img, images, num_augmentations)
        try:
            add_item_to_hdf5(filename, result) # future.result())
            # for pair in future.result():
            #     arr1, arr2, label = pair
            #     group = hf.create_group(f'item_{i}')
            #     group.create_dataset('image1', data=arr1, compression="gzip", compression_opts=9)
            #     group.create_dataset('image2', data=arr2, compression="gzip", compression_opts=9)
            #     group.attrs['label'] = label
            #     i += 1
        except Exception as e:
            logging.error(f"Error in future result: {e}")

split = 'train'

with h5py.File(f'data/CS_Detector/{split}_pairs.h5', 'w') as hf:
    # START = len(hf.keys())
    # print(START)
    ...

generate_pairs(split, data_splits)

In [ ]:
with h5py.File(f'data/CS_Detector/train_pairs.h5', 'r') as hf:
    print(len(hf.keys()))

## Create Embeddings

In [ ]:
import h5py
from PIL import Image
import torch
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

def add_item_to_hdf5(filename, new_data):
    with h5py.File(filename, 'a') as hf:
        # Determine the next available index for the new item
        item_index = len(hf.keys())

        for pair in new_data:
            arr1, arr2, label = pair
            group = hf.create_group(f'item_{item_index}')
            group.create_dataset('image1', data=arr1, compression="gzip", compression_opts=9)
            group.create_dataset('image2', data=arr2, compression="gzip", compression_opts=9)
            group.attrs['label'] = label
            item_index += 1

def split_image_into_squares(img, overlap_ratio=0.25):
    img = Image.fromarray(img.astype(np.uint8))
    width, height = img.size
    square_side = min(width, height)
    step_size = int(square_side * (1 - overlap_ratio))  # Step size adjusted for overlap

    squares = []
    if width > height:
        # Process width for overlaps
        for start in range(0, width - square_side + 1, step_size):
            square = img.crop((start, 0, start + square_side, height))
            squares.append(square)
        # Add the last piece if needed
        if start + square_side < width:
            square = img.crop((width - square_side, 0, width, height))
            squares.append(square)
    else:
        # Process height for overlaps
        for start in range(0, height - square_side + 1, step_size):
            square = img.crop((0, start, width, start + square_side))
            squares.append(square)
        # Add the last piece if needed
        if start + square_side < height:
            square = img.crop((0, height - square_side, width, height))
            squares.append(square)

    return squares

## CLIP

In [ ]:
save_dir = "data/CS_Detector/CLIP"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
try:
    import clip
except:
    !pip install git+https://github.com/openai/CLIP.git -q
    import clip

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 66.8 MB/s eta 0:00:00


In [ ]:
# Load the CLIP model and the preprocess function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, preprocess = clip.load('ViT-L/14@336px', device=device)

100%|███████████████████████████████████████| 891M/891M [00:16<00:00, 57.1MiB/s]


#### Generate Data

In [ ]:
def save_embeddings(split):
    # Download the image

    assert os.path.exists(f"{save_dir}/{split}_pairs.h5") == False, "You risk overwriting this file..."

    with h5py.File(f"{save_dir}/{split}_pairs.h5", 'w') as hf:
        ...

    with h5py.File(f'data/CS_Detector/{split}_pairs.h5', 'r') as hf:
        for item in tqdm(hf.keys()):
            group = hf[item]
            image1 = np.array(group['image1'])
            image2 = np.array(group['image2'])
            label = group.attrs['label']

            images1 = split_image_into_squares(image1, 0.22)
            images2 = split_image_into_squares(image2, 0.22)


            data = []
            for img1, img2 in zip(images1, images2):
                img1 = preprocess(img1).unsqueeze(0).to(device)
                img2 = preprocess(img2).unsqueeze(0).to(device)
                with torch.no_grad():
                    image_features1 = model.encode_image(img1)
                    image_features2 = model.encode_image(img2)

                image_features1 = image_features1.detach().cpu().numpy()
                image_features2 = image_features2.detach().cpu().numpy()
                data.append([image_features1, image_features2, label])

            add_item_to_hdf5(f"{save_dir}/{split}_pairs.h5", data)

save_embeddings('train')

  0%|          | 0/60179 [00:00<?, ?it/s]

In [ ]:
split = 'train'
mode = 'r'
filename = f"{save_dir}/{split}_pairs.h5"
if mode == 'w':
    assert os.path.exists(filename) == True, "You risk overwriting this file..."

with h5py.File(filename, mode) as hf:
    print(len(hf.keys()))

120358


In [ ]:
!ls data/CS_Detector/CLIP

test_pairs.h5  train_pairs_first_third.h5  train_pairs.h5  val_pairs.h5


In [ ]:
def save_embeddings(split):
    # Download the image

    filename = f"{save_dir}/{split}_pairs.h5"

    if not os.path.exists(filename):
        with h5py.File(filename, 'w') as hf:
            ...

    with h5py.File(f'data/CS_Detector/{split}_pairs.h5', 'r') as hf:
        for item in tqdm(items[40000:]):
            group = hf[item]
            image1 = np.array(group['image1'])
            image2 = np.array(group['image2'])
            label = group.attrs['label']

            images1 = split_image_into_squares(image1, 0.22)
            images2 = split_image_into_squares(image2, 0.22)


            data = []
            for img1, img2 in zip(images1, images2):
                img1 = preprocess(img1).unsqueeze(0).to(device)
                img2 = preprocess(img2).unsqueeze(0).to(device)
                with torch.no_grad():
                    image_features1 = model.encode_image(img1)
                    image_features2 = model.encode_image(img2)

                image_features1 = image_features1.detach().cpu().numpy()
                image_features2 = image_features2.detach().cpu().numpy()
                data.append([image_features1, image_features2, label])

            add_item_to_hdf5(filename, data)

save_embeddings('train')

  0%|          | 0/20179 [00:00<?, ?it/s]

## MetaCLIP

In [ ]:
save_dir = "data/CS_Detector/MetaCLIP"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
!pip install open_clip_torch -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 39.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 94.1 MB/s eta 0:00:00


In [ ]:
import open_clip
import torch
from PIL import Image

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model, _, preprocess = open_clip.create_model_and_transforms('ViT-H-14-quickgelu', pretrained='metaclip_fullcc', device=device)  # for 2.5B use 'metaclip_fullcc' in OpenCLIP or 'metaclip_2_5b' in this repo

100%|██████████████████████████████████████| 11.8G/11.8G [00:54<00:00, 216MiB/s]


#### Generate Data

In [ ]:
!ls data/CS_Detector/MetaCLIP

test_pairs.h5  train_pairs.h5  val_pairs.h5


In [ ]:
def save_meta_embeddings(split, mode='w'):
    # Download the image

    filename = f"{save_dir}/{split}_pairs.h5"

    assert not (os.path.exists(filename) and mode == 'w'), 'You risk overwriting the file.'

    with h5py.File(filename, mode) as hfw:
        item_index = len(hfw.keys())
        print("Starting at: ", item_index)
        with h5py.File(f'data/CS_Detector/{split}_pairs.h5', 'r') as hf:
            for item in tqdm(list(hf.keys())[40_000:]):
                group = hf[item]
                image1 = np.array(group['image1'])
                image2 = np.array(group['image2'])
                label = group.attrs['label']

                images1 = split_image_into_squares(image1, 0.22)
                images2 = split_image_into_squares(image2, 0.22)


                for img1, img2 in zip(images1, images2):
                    img1 = preprocess(img1).unsqueeze(0).to(device)
                    img2 = preprocess(img2).unsqueeze(0).to(device)
                    with torch.no_grad():
                        image_features1 = model.encode_image(img1)
                        image_features2 = model.encode_image(img2)

                    image_features1 = image_features1.detach().cpu().numpy()
                    image_features2 = image_features2.detach().cpu().numpy()

                    group = hfw.create_group(f'item_{item_index}')
                    group.create_dataset('image1', data=image_features1, compression="gzip", compression_opts=9)
                    group.create_dataset('image2', data=image_features2, compression="gzip", compression_opts=9)
                    group.attrs['label'] = label
                    item_index += 1

save_meta_embeddings('train', 'a')

Starting at:  80000


  0%|          | 0/20179 [00:00<?, ?it/s]

# Preparing data for SSL with TF Similarity